In [26]:
import open3d as o3d
import numpy as np

filepath = './Demo/Lloyd PC.ply'
pcd = o3d.io.read_point_cloud(filepath)

In [ ]:
#pcd.paint_uniform_color([0.4,0.9,0.2])
#o3d.visualization.draw_geometries([pcd])

In [27]:
print(pcd)
print(np.asarray(pcd.points))

PointCloud with 1003510 points.
[[-0.65949464  0.83651191 -0.83541268]
 [-0.12086892  0.04687786 -0.55958313]
 [-0.03233266 -0.67470443 -0.69881791]
 ...
 [-0.31737113  0.47792697 -0.69495457]
 [-0.5055514   0.23335868 -0.68829918]
 [-0.07477051 -0.05247402  0.08733849]]


In [28]:
#Floor remover: find plane and cut everything below it
plane_model, inliers = pcd.segment_plane(distance_threshold=0.003,
                                         ransac_n=5,
                                         num_iterations=2000)

inlier_cloud = pcd.select_by_index(inliers)
inlier_cloud.paint_uniform_color([1.0, 0, 0])
#box = inlier_cloud.get_axis_aligned_bounding_box() #Creates bounding box for point cloud
#bounds = np.asarray(box.get_box_points()) #Get points for bounding box
#bounds = bounds[bounds[:,2].argsort()]
#for i in range(4):
#    bounds[i][2] = pcd.get_min_bound()[2]
#cpcd = pcd.crop(o3d.geometry.AxisAlignedBoundingBox.create_from_points(o3d.utility.Vector3dVector(bounds)), invert=True)

PointCloud with 365276 points.

In [29]:
box = pcd.get_axis_aligned_bounding_box() #Creates bounding box for point cloud
inlierbox = inlier_cloud.get_axis_aligned_bounding_box()
bounds = np.asarray(box.get_box_points()) #Get points for bounding box
inlierbounds = np.asarray(inlierbox.get_box_points())
bounds = bounds[bounds[:,2].argsort()]
inlierbounds = inlierbounds[inlierbounds[:,2].argsort()]

for i in range(4):
    bounds[(i+4)][2]= inlierbounds[(i+4)][2]
cpcd = pcd.crop(o3d.geometry.AxisAlignedBoundingBox.create_from_points(o3d.utility.Vector3dVector(bounds)), invert=True)

In [ ]:
cpcd, ind = cpcd.remove_statistical_outlier(nb_neighbors=15,
                                                    std_ratio=2.0)
print(cpcd)

In [ ]:
cpcd, ind = cpcd.remove_statistical_outlier(nb_neighbors=20,
                                                    std_ratio=1.7)
print(cpcd)

In [ ]:
cpcd, ind = cpcd.remove_statistical_outlier(nb_neighbors=25,
                                                    std_ratio=1.7)
print(cpcd)

In [30]:
o3d.visualization.draw_geometries([cpcd])

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 


In [31]:
cpcd.estimate_normals(
    search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.5, max_nn=30))
o3d.visualization.draw_geometries([cpcd],
                                  point_show_normal=True)

In [32]:
d = 7
pmesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(cpcd, depth=d)

In [33]:
print(pmesh)

TriangleMesh with 6379 points and 12666 triangles.


In [34]:
pmesh.paint_uniform_color([0.1,0.7,0.5])
o3d.visualization.draw_geometries([pmesh], mesh_show_wireframe=True)